In [1]:
import pandas as pd
import numpy as np
import os
import category_encoders as ce

In [2]:
%%time 

df = pd.read_parquet('clean-data.parquet', engine='pyarrow')

CPU times: total: 688 ms
Wall time: 434 ms


In [3]:
df = df.reset_index(drop=True)
df

,Hospital Service Area,Hospital County,Operating Certificate Number,Age Group,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,CCSR Diagnosis Code,CCSR Procedure Code,APR Severity of Illness Code,APR Risk of Mortality,Payment Typology 1,Total Charges,Total Costs
0,New York City,Bronx,7000014.0,0 to 17,M,White,Not Span/Hispanic,3,Emergency,Home or Self Care,A78,A030,2,Minor,Self-Pay,13697.82,3238.19
1,New York City,Kings,7001017.0,0 to 17,M,White,Not Span/Hispanic,2,Newborn,Home or Self Care,D250,D230,3,Minor,Insurance Company,7271.95,3856.94
2,New York City,Manhattan,7002020.0,0 to 17,M,White,Not Span/Hispanic,1,Emergency,Home or Self Care,C228,D020,1,Moderate,Blue Cross,6253.88,2923.55
3,New York City,Manhattan,7002002.0,0 to 17,M,White,Not Span/Hispanic,3,Emergency,Home or Self Care,A78,D020,3,Minor,Insurance Company,16128.93,8410.92
4,New York City,Manhattan,7002017.0,0 to 17,M,White,Not Span/Hispanic,4,Newborn,Home or Self Care,D250,D020,1,Minor,Insurance Company,12059.28,1362.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743774,Long Island,Nassau,2953000.0,70 or Older,F,Other Race,Not Span/Hispanic,3,Emergency,Home or Self Care,C183,B150,2,Moderate,Medicare,26075.00,7190.31
743775,Long Island,Nassau,2953000.0,70 or Older,F,Other Race,Not Span/Hispanic,10,Emergency,Home w/ Home Health Services,B830,B000,3,Major,Medicare,69555.50,20170.58
743776,Long Island,Suffolk,5123000.0,70 or Older,F,Other Race,Unknown,7,Emergency,Home or Self Care,C170,B509,2,Minor,Medicare,76299.60,11852.36
743777,Long Island,Suffolk,5123000.0,70 or Older,F,Other Race,Not Span/Hispanic,3,Emergency,Home or Self Care,E060,C7250,1,Minor,Medicare,24812.95,4361.88


# Encoding Scheme

## Hospital Information

- **Operating Certificate Number - Hamiltonian path (see operating-geo.csv)** 

- **Hospital Service Area - geographic Hamiltonian path (pasted below)**
  
```python
service_area_mapping = {
    'Western NY': 1,
    'Finger Lakes': 2,
    'Southern Tier': 3,
    'Central NY': 4,
    'Capital/Adiron': 5,
    'Hudson Valley': 6,
    'New York City': 7,
    'Long Island': 8
}
```

- **Hospital County - geographic Hamiltonian path (pasted below)**
       
```python
county_mapping = {
    'Chautauqua': 1,
    'Cattaraugus': 2,
    'Erie': 3,
    'Niagara': 4,
    'Orleans': 5,
    'Genesee': 6,
    'Wyoming': 7,
    'Allegany': 8,
    'Monroe': 9,
    'Wayne': 10,
    'Livingston': 11,
    'Ontario': 12,
    'Yates': 13,
    'Steuben': 14,
    'Schuyler': 15,
    'Chemung': 16,
    'Tompkins': 17,
    'Cortland': 18,
    'Cayuga': 19,
    'Onondaga': 20,
    'Oswego': 21,
    'Jefferson': 22,
    'St Lawrence': 23,
    'Lewis': 24,
    'Franklin': 25,
    'Clinton': 26,
    'Essex': 27,
    'Warren': 28,
    'Saratoga': 29,
    'Schenectady': 30,
    'Albany': 31,
    'Rensselaer': 32,
    'Montgomery': 33,
    'Fulton': 34,
    'Schoharie': 35,
    'Herkimer': 36,
    'Oneida': 37,
    'Madison': 38,
    'Chenango': 39,
    'Broome': 40,
    'Otsego': 41,
    'Delaware': 42,
    'Sullivan': 43,
    'Orange': 44,
    'Putnam': 45,
    'Dutchess': 46,
    'Columbia': 47,
    'Ulster': 48,
    'Rockland': 49,
    'Westchester': 50,
    'Bronx': 51,
    'Manhattan': 52,
    'Kings': 53,
    'Queens': 54,
    'Richmond': 55,
    'Nassau': 56,
    'Suffolk': 57
}
```

## Patient Demographics

- **Age Group - ordinal**

```python 
age_group_mapping = {
    '0 to 17': 0,
    '18 to 29': 1,
    '30 to 49': 2,
    '50 to 69': 3,
    '70 or Older': 4
}
```

- **Gender - one-hot**
- **Race - one-hot**
- **Ethnicity - one-hot**

## Hospital Stay

- **Length of Stay - ordinal (120+ string to 120 int)**
- **Type of Admission - one-hot (dropping 'Not Available' column)**
- **Patient Disposition - ordinal in the following order:**

```python 
patient_disposition_mapping = {
    'Home or Self Care': 1,
    'Admitted from Ambulatory Surgery': 2,
    'Home w/ Home Health Services': 3,
    'Hospice - Home': 4,
    'Facility w/ Custodial/Supportive Care': 5,
    'Skilled Nursing Home': 6,
    'Medicaid Cert Nursing Facility': 7,
    'Medicare Cert Long Term Care Hospital': 8,
    'Inpatient Rehabilitation Facility': 9,
    'Critical Access Hospital': 10,
    'Hosp Basd Medicare Approved Swing Bed': 11,
    "Cancer Center or Children's Hospital": 12,
    'Federal Health Care Facility': 13,
    'Short-term Hospital': 14,
    'Psychiatric Hospital or Unit of Hosp': 15,
    'Another Type Not Listed': 16,
    'Left Against Medical Advice': 17,
    'Court/Law Enforcement': 18,
    'Hospice - Medical Facility': 19,
    'Expired': 20
}
```

## Medical Information

- **CCSR Diagnosis Code - alphabetic sorted ordinal**
- **CCSR Procedure Code - alphabetic sorted ordinal**

## Severity & Risk

- **APR Severity of Illness Code - one-hot (drop 0)**
- **APR Risk of Mortality - one-hot**

## Financial Information

- **Payment Typology 1 - one-hot**
- **Total Charges - already numeric**
- **Total Costs - already numeric**

In [4]:
# Define the categories and their respective columns
categories = {
    'Hospital Information': ['Operating Certificate Number', 'Hospital Service Area', 'Hospital County', 'Facility Name'],
    'Patient Demographics': ['Age Group', 'Gender', 'Race', 'Ethnicity'],
    'Hospital Stay': ['Length of Stay', 'Type of Admission', 'Patient Disposition'],
    'Medical Information': ['CCSR Diagnosis Code', 'CCSR Procedure Code'],
    'Severity & Risk': ['APR Severity of Illness Code', 'APR Risk of Mortality'],
    'Financial Information': ['Payment Typology 1', 'Total Charges', 'Total Costs']
}

# Initialize an empty DataFrame to hold the final results
final_df = pd.DataFrame(columns=['Category', 'Year', 'Unique Values', 'Number of Unique Values', 'Percentage of Null Values'])

# Define the year - you can change this as needed
year = '2009'

# Iterate over each category and column
for category, columns in categories.items():
    for col in columns:
        if col in df.columns:
            unique_values = df[col].unique()
            num_unique_values = df[col].nunique()
            percent_null_values = (df[col].isna().sum() / len(df)) * 100

            # Create a temporary DataFrame to hold this information
            temp_df = pd.DataFrame({
                'Category': [category],
                'Year': [year],
                'Unique Values': [unique_values],
                'Number of Unique Values': [num_unique_values],
                'Percentage of Null Values': [percent_null_values]
            }, index=[col])  # Set the index to the column name

            # Append this information to the final DataFrame
            final_df = pd.concat([final_df, temp_df])

# Create a multi-index DataFrame from the final_df
final_df.reset_index(inplace=True)
final_df.set_index(['index', 'Year'], inplace=True)  # Multi-index using the column name and year
final_df.sort_index(inplace=True)

# final_df is your final DataFrame with the desired format

C:\Users\Windows\AppData\Local\Temp\ipykernel_1100\377971296.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, temp_df])


In [5]:
def display_by_category(final_df, category):
    """
    Display a filtered version of final_df based on the given category, omitting the 'Category' column.
    
    Parameters:
    - final_df: DataFrame, the multi-index DataFrame containing the aggregate info
    - category: str, the category to filter by
    """
    # Filter the DataFrame based on the 'Category' column
    filtered_df = final_df[final_df['Category'] == category]
    
    # Drop the 'Category' column
    filtered_df = filtered_df.drop(columns=['Category'])
    
    return filtered_df

In [6]:
def find_unique(df, column_name):
    """Finds unique values for a specified column from a dataframe."""
    # Find unique values in the given column
    unique_values = df[column_name].unique().tolist()
    
    return unique_values

## Hospital Information

In [7]:
hospital_info_df = display_by_category(final_df, 'Hospital Information')
print("Hospital Information:")
hospital_info_df

Hospital Information:


,,Unique Values,Number of Unique Values,Percentage of Null Values
index,Year,,,
Hospital County,2009,"[Bronx, Kings, Manhattan, Westchester, Queens,...",57,0.0
Hospital Service Area,2009,"[New York City, Hudson Valley, Finger Lakes, L...",8,0.0
Operating Certificate Number,2009,"[7000014.0, 7001017.0, 7002020.0, 7002002.0, 7...",183,0.0


### Hamiltonian path on 'Operating Certificate Number' (Optional)

### Hamiltonian path on 'Hospital Service Area' (Optional)

### Hamiltonian path on 'Hospital County' (Optional)

## Patient Demographics

### Ordinal Encoding on Age Group

In [8]:
unique_age_groups = find_unique(df, "Age Group")
print(f'{len(unique_age_groups)} unique "Age Group" values')
unique_age_groups

5 unique "Age Group" values


['0 to 17', '18 to 29', '30 to 49', '50 to 69', '70 or Older']

In [9]:
# Define ordinal mapping for the 'Age Group' column
age_group_mapping = {
    '0 to 17': 1,
    '18 to 29': 2,
    '30 to 49': 3,
    '50 to 69': 4,
    '70 or Older': 5
}

# Function to encode 'Age Group' in the DataFrame
def encode_age_group(df, mapping):
    if 'Age Group' in df.columns:
        df['Age Group'] = df['Age Group'].map(mapping)
        
        # Check for NaN values and issue a warning if any are found
        if df['Age Group'].isna().any():
            print("Warning: NaN values found, not all entries could be mapped.")
    return df

# Usage
df = encode_age_group(df, age_group_mapping)


### One-hot Encoding on Ethnicity 

In [10]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['Ethnicity'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

### One-hot Encoding on Gender

In [11]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['Gender'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

### One-hot Encoding on Race

In [12]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['Race'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

# Hospital Stay

In [13]:
hospital_stay_df = display_by_category(final_df, 'Hospital Stay')
print("Hospital Stay:")
hospital_stay_df

Hospital Stay:


,,Unique Values,Number of Unique Values,Percentage of Null Values
index,Year,,,
Length of Stay,2009,"[3, 2, 1, 4, 8, 6, 16, 7, 23, 5, 22, 15, 78, 1...",120,0.0
Patient Disposition,2009,"[Home or Self Care, Short-term Hospital, Cance...",19,0.0
Type of Admission,2009,"[Emergency, Newborn, Elective, Urgent, Not Ava...",6,0.0


## Oridinal Encoding on Length of Stay

In [14]:
unique_length_of_stays = find_unique(df, "Length of Stay")
print(f'{len(unique_length_of_stays)} unique "Length of Stay" values')
unique_length_of_stays

120 unique "Length of Stay" values


['3',
 '2',
 '1',
 '4',
 '8',
 '6',
 '16',
 '7',
 '23',
 '5',
 '22',
 '15',
 '78',
 '10',
 '24',
 '18',
 '20',
 '59',
 '9',
 '13',
 '17',
 '11',
 '14',
 '12',
 '32',
 '36',
 '21',
 '40',
 '19',
 '25',
 '30',
 '35',
 '31',
 '58',
 '26',
 '28',
 '45',
 '34',
 '29',
 '120 +',
 '43',
 '80',
 '67',
 '103',
 '44',
 '73',
 '64',
 '49',
 '47',
 '60',
 '115',
 '57',
 '46',
 '83',
 '27',
 '76',
 '112',
 '38',
 '48',
 '37',
 '50',
 '52',
 '39',
 '77',
 '65',
 '100',
 '118',
 '55',
 '54',
 '90',
 '111',
 '56',
 '70',
 '79',
 '92',
 '33',
 '91',
 '88',
 '74',
 '89',
 '85',
 '98',
 '62',
 '63',
 '51',
 '71',
 '72',
 '105',
 '42',
 '61',
 '41',
 '95',
 '75',
 '84',
 '53',
 '66',
 '101',
 '94',
 '87',
 '114',
 '69',
 '68',
 '99',
 '107',
 '81',
 '110',
 '106',
 '96',
 '108',
 '109',
 '113',
 '97',
 '82',
 '102',
 '86',
 '104',
 '119',
 '93',
 '116',
 '117']

In [15]:
def encode_length_of_stay(df):
    if 'Length of Stay' in df.columns:
        
        # Replace '120 +' with 120 for conversion to integer
        df['Length of Stay'].replace('120 +', '120', inplace=True)
        
        # Convert the column to integer type for ordinal nature
        df['Length of Stay'] = df['Length of Stay'].astype(int)
        
        # Sort the unique values in ascending order
        sorted_unique_values = sorted(df['Length of Stay'].unique())
        
        # Create an ordinal mapping from sorted unique values
        ordinal_mapping = {val: idx + 1 for idx, val in enumerate(sorted_unique_values)}
        
        # Map the column to its ordinal encoding
        df['Length of Stay'] = df['Length of Stay'].map(ordinal_mapping)
        
        # Check for NaN values and issue a warning if any are found
        if df['Length of Stay'].isna().any():
            print("Warning: NaN values found, not all entries could be mapped.")
            
    return df

df = encode_length_of_stay(df)

## One-hot Encoding on Type of Admission

In [16]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['Type of Admission'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

## Ordinal Encoding on Patient Disposition

In [17]:
# Create a mapping dictionary for 'Patient Disposition'
patient_disposition_mapping = {
    'Home or Self Care': 1,
    'Admitted from Ambulatory Surgery': 2,
    'Home w/ Home Health Services': 3,
    'Hospice - Home': 4,
    'Facility w/ Custodial/Supportive Care': 5,
    'Skilled Nursing Home': 6,
    'Medicaid Cert Nursing Facility': 7,
    'Medicare Cert Long Term Care Hospital': 8,
    'Inpatient Rehabilitation Facility': 9,
    'Critical Access Hospital': 10,
    'Hosp Basd Medicare Approved Swing Bed': 11,
    "Cancer Center or Children's Hospital": 12,
    'Federal Health Care Facility': 13,
    'Short-term Hospital': 14,
    'Psychiatric Hospital or Unit of Hosp': 15,
    'Another Type Not Listed': 16,
    'Left Against Medical Advice': 17,
    'Court/Law Enforcement': 18,
    'Hospice - Medical Facility': 19,
    'Expired': 20
}

# Function to apply ordinal encoding to the 'Patient Disposition' column
def ordinal_encode_column(df, column_name, mapping_dict):
    if column_name in df.columns:
        df[column_name] = df[column_name].map(mapping_dict)
    return df

df = ordinal_encode_column(df, 'Patient Disposition', patient_disposition_mapping)

# Medical Information

In [18]:
medical_info_df = display_by_category(final_df, 'Medical Information')
print("Medical Information:")
medical_info_df

Medical Information:


,,Unique Values,Number of Unique Values,Percentage of Null Values
index,Year,,,
CCSR Diagnosis Code,2009,"[A78, D250, C228, A840, A3700, A360, D3140, E2...",166,0.0
CCSR Procedure Code,2009,"[A030, D230, D020, D3000, D3140, A65, B3783, A...",152,0.0


### Oridnal Encoding on CCSR Diagnosis Code (Optional)

### Ordinal Encoding on CCSR Procedure Code (Optional)

# Severity & Risk

In [19]:
severity_and_risk_df = display_by_category(final_df, 'Severity & Risk')
print("Severity & Risk:")
severity_and_risk_df

Severity & Risk:


,,Unique Values,Number of Unique Values,Percentage of Null Values
index,Year,,,
APR Risk of Mortality,2009,"[Minor, Moderate, Major, Extreme]",4,0.0
APR Severity of Illness Code,2009,"[2, 3, 1, 4]",4,0.0


### One-hot Encoding on APR Severity of Illness Code

In [20]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['APR Severity of Illness Code'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

### One-hot Encoding on APR Risk of Mortality

In [21]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['APR Risk of Mortality'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

# Financial Information

In [22]:
financial_info_df = display_by_category(final_df, 'Financial Information')
print("Financial Information:")
financial_info_df

Financial Information:


,,Unique Values,Number of Unique Values,Percentage of Null Values
index,Year,,,
Payment Typology 1,2009,"[Self-Pay, Insurance Company, Blue Cross, Medi...",10,0.0
Total Charges,2009,"[13697.82, 7271.95, 6253.88, 16128.93, 12059.2...",631152,0.0
Total Costs,2009,"[3238.19, 3856.94, 2923.55, 8410.92, 1362.99, ...",602461,0.0


In [23]:
unique_payments = find_unique(df, "Payment Typology 1")
print(f'{len(unique_payments)} unique "Payment Typology 1" values')
unique_payments

10 unique "Payment Typology 1" values


['Self-Pay',
 'Insurance Company',
 'Blue Cross',
 'Medicaid',
 'Other Federal Program',
 'CHAMPUS',
 'Medicare',
 'Workers Compensation',
 'Other Non-Federal Program',
 'Unknown']

### One-hot Encoding on Payment Typology 1

In [24]:
# Define the encoder
encoder = ce.OneHotEncoder(cols=['Payment Typology 1'], use_cat_names=True)

# Fit and transform the DataFrame
df = encoder.fit_transform(df)

C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\utils.py:50: FutureWarni

### Total Charges (no encoding needed)

In [25]:
unique_charges = find_unique(df, "Total Charges")
print(f'{len(unique_charges)} unique "Total Charges" values')
#unique_charges

all_floats = all(isinstance(x, float) for x in unique_charges)
print("All elements are floats:", all_floats)

631152 unique "Total Charges" values
All elements are floats: True


### Total Costs (no encoding needed)

In [26]:
unique_costs = find_unique(df, "Total Costs")
print(f'{len(unique_costs)} unique "Total Costs" values')
#unique_costs

all_floats = all(isinstance(x, float) for x in unique_costs)
print("All elements are floats:", all_floats)

602461 unique "Total Costs" values
All elements are floats: True


# Final Check & Saving

In [27]:
num_rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"Number of rows with null values: {num_rows_with_nulls}")

Number of rows with null values: 0


In [28]:
# Saving DataFrame to Parquet format
df.to_parquet('ny-final.parquet')